In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import itertools

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Prep

In [35]:
NUM_SYMBOLS = 7
permutations = [''.join(x) for x in itertools.permutations([str(i+1) for i in range(NUM_SYMBOLS)], NUM_SYMBOLS)]
len(permutations)
data = pd.DataFrame(permutations, columns=['perm']).sort_values(by='perm').reset_index(drop=True)
data['roll_grp'] = data.perm.apply(lambda x: x[x.find('1'):]+x[:x.find('1')])
data['roll_val'] = data.perm+data.perm.str[:-1]

data

In [36]:
overlaps = [(i+1) for i in range(0, NUM_SYMBOLS-2)]
overlaps.reverse()
overlaps

# Generate SuperPerms

In [37]:
def gen_code(av_data = data.copy(), gen_code_grp = True):
   
    code = ''
    num_found=0

    while av_data.shape[0]>0:
        for idx, row in av_data.iterrows():
            found=False

            if code == '':
                code = row.roll_val
                found=True
                num_found +=1
                data.loc[data.roll_grp==row.roll_grp, 'code_grp']=0
                av_data = av_data[av_data.roll_grp!=row.roll_grp]
                break
            else:
                for ov in overlaps:
                    for _, row2 in av_data.iterrows():
                        if code[-ov:] == row2.roll_val[:ov]:
                            v_last_ov = ov
                            code += row2.roll_val[ov:]
                            found=True
                            break
                    if found:
                        break
                if found:
                    num_found +=1
                    
                    # put processed permutation in one of three groups
                    if gen_code_grp:
                        if num_found <= data.roll_grp.drop_duplicates().shape[0]//3:
                            data.loc[data.roll_grp==row2.roll_grp, 'code_grp']=0
                        elif num_found <= data.roll_grp.drop_duplicates().shape[0]//3*2:
                            data.loc[data.roll_grp==row2.roll_grp, 'code_grp']=1
                        else:
                            data.loc[data.roll_grp==row2.roll_grp, 'code_grp']=2

                    av_data = av_data[av_data.roll_grp!=row2.roll_grp]
                    break

    print(f'The generated string containing all permutations has {len(code)} characters.\n')

    return code

In [38]:
%%time

code = gen_code(av_data = data.copy(), gen_code_grp = True)

# Split data and extend for competition

In [39]:
lc = len(code)

fin_queues = [None]*3

fin_queues[0] = code[:(lc//3+6)]
fin_queues[1] = code[(lc//3):-(lc//3-6)]
fin_queues[2] = code[-(lc//3+6):]

max_len = 0
for it, q in enumerate(fin_queues):
    max_len = max(max_len, len(q))
    print(f'String {it} contains {len(q)} characters')

print(f'\nThe longest String contains {max_len} characters')

In [40]:
%%time
fin_queues = [None]*3
for it in range(3):
    fin_queues[it] = gen_code(av_data = data[data.code_grp==it].copy(), gen_code_grp = False)
    
max_len = 0
for it, q in enumerate(fin_queues):
    max_len = max(max_len, len(q))
    print(f'String {it} contains {len(q)} characters')

print(f'\nThe longest String contains {max_len} characters')

# Check for missing perms

In [41]:
i=0
for p in permutations:
    if (p in fin_queues[0])|(p in fin_queues[1])|(p in fin_queues[2]):
        pass 
    else:
        print(p)
        i+=1
assert(i == 0)

# Find Best Santa Coupling

In [42]:
perm_santa_couple = [''.join(x) for x in itertools.permutations([str(i+1) for i in range(NUM_SYMBOLS)], 2)]

for ps in perm_santa_couple:
    log = f'Santas: {ps}\n'
    
    avg = 0
    mn = 9999
    for qn, q in enumerate(fin_queues):
        avg += q.count(ps)/3
        mn = min(mn, q.count(ps))
        log += f'counts in Q({qn}):{q.count(ps)}\n'
    log+=f'avg: {avg:.2f}, min: {mn}\n'
    
    if (mn >= 50)&(avg>=50):
        print(log)
        
santa_couple_code = '35'

def fill_q(fin_queues):
    santa_couple_add = 0
    for q_id in range(3):
        print(f'queue({q_id})#')
        print(f'length before santa couple added: {len(fin_queues[q_id])}')
        for perm in data[data.perm.str.startswith(santa_couple_code)].perm.values:
            if perm not in fin_queues[q_id]:
                fin_queues[q_id] = fin_queues[q_id]+perm
                santa_couple_add +=1
        print(f'length after: {len(fin_queues[q_id])}\n')
    
    print(f'SantasCoder to add: {santa_couple_add}\n')
    print(f'* {len(fin_queues[0]),len(fin_queues[1]),len(fin_queues[2])}\n')
    return fin_queues
fin_queues_ext = fill_q(fin_queues)

# Prep for Submission

In [43]:
replace_dict = {
    "4": '🎅', 
    "5": '🤶', 
    "3": '🦌', 
    "1": '🧝', 
    "2": '🎄', 
    "6": '🎁', 
    "7": '🎀', 
}

ans = fin_queues_ext.copy()
for i in range(3):
    for k,v in replace_dict.items():
        ans[i] = ans[i].replace(k, v)
ans[0][:20]
sub = pd.DataFrame()
sub['schedule'] = [ans[i] for i in range(3)]

sub
sub.to_csv('submission2.csv', index = False)